In [1]:
import os
import csv
!pip install nlpaug


stop_words = ['the', 'and', 'or', 'is', 'a', 'an', 'of', 'in', 'to', 'for', 'with', 'on',"/","~","`","!","@","#","$","%","^","&","*","()","(",")","<",">","'",":",'"']
import string


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 4.5 MB/s eta 0:00:00


In [2]:
#prepocess dataset

#!git clone "https://github.com/google-research-datasets/poem-sentiment/"
!git clone "https://huggingface.co/datasets/mteb/tweet_sentiment_extraction/"
!gdown "1nOq-QxagdrNYag51XDhQwTCTok6LN89h" #sentimemt140

!gdown "1XtKRlTEjPN-Yp65cZKuiEcGxzr4HCY8D" #yg lama

!gdown "1iEQeMmeOp0UQVSdbKI3JBw4TOWPQULy-" #tambahan netral

!gdown "1CsrcLUqX9kI4L_qG_iwtamVeCgAVQB1i" #tambahan negatif


Cloning into 'poem-sentiment'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 22 (delta 2), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (22/22), 33.54 KiB | 4.19 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Cloning into 'tweet_sentiment_extraction'...
remote: Enumerating objects: 15, done.
remote: Total 15 (delta 0), reused 0 (delta 0), pack-reused 15
Unpacking objects: 100% (15/15), 1.31 MiB | 7.28 MiB/s, done.
Downloading...
From: https://drive.google.com/uc?id=1nOq-QxagdrNYag51XDhQwTCTok6LN89h
To: /content/sentiment140.zip
100% 84.9M/84.9M [00:01<00:00, 52.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1XtKRlTEjPN-Yp65cZKuiEcGxzr4HCY8D
To: /content/MH_Campaign_Tweets_Sentiment_Scored_1723.csv
100% 735M/735M [00:10<00:00, 69.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1iEQeMmeOp0UQVSdbKI3JBw4TOWPQULy-
To: /content/tambahan_netral.txt
100%

In [3]:
!pip install nlpaug
!unzip "/content/sentiment140.zip"

Archive:  /content/sentiment140.zip
  inflating: training.1600000.processed.noemoticon.csv  


In [ ]:
import csv
import re
import ast
import nlpaug.augmenter.word as aug
import string
import numpy as np
import random
import json

stop_words = ['the', 'and', 'or', 'is', 'a', 'an', 'of', 'in', 'to', 'for', 'with', 'on',"/"," /"," / ","/ ","~","`","!","@","#","$","%","^","&","*","()","(",")","<",">","'",":",'"','/',"http","sunny","com"]
path1 = "/content/training.1600000.processed.noemoticon.csv"
path2 = "/content/MH_Campaign_Tweets_Sentiment_Scored_1723.csv"
path3 = "/content/tambahan_netral.txt"
path4 = "/content/tambahan_negatif.txt"
path5 = "/content/tweet_sentiment_extraction/train.jsonl"
path6 = "/content/tweet_sentiment_extraction/test.jsonl"

tulis_path = "/content/drive/MyDrive/DATASET_CAPSTONE/full_AUG.csv"

kalimat_neg = []
score_neg = []

kalimat_neu = []
score_neu = []

kalimat_pos = []
score_pos = []

jumlah = [0, 0, 0]

# Preprocess the first dataset
with open(path1, "r", encoding="latin-1") as data:
    file = csv.reader(data, delimiter=",", quotechar='"')
    next(file)
    for j in file:
        try:
            lab = int(j[0])
            text = re.sub(r'[-.,:;!?\n@#]|//t', ' ', j[5])
            text = ' '.join([word.strip(string.punctuation) for word in text.split() if word.lower() not in stop_words])

            if lab == 0:
                score_neg.append([1, 0, 0])
                jumlah[0] += 1
                kalimat_neg.append(text)
            elif lab == 2:
                score_neu.append([0, 1, 0])
                jumlah[1] += 1
                kalimat_neu.append(text)
            elif lab == 4:
                score_pos.append([0, 0, 1])
                jumlah[2] += 1
                kalimat_pos.append(text)
        except Exception as e:
            print("error:", e)

print("First dataset processed:", jumlah)

# Preprocess the second dataset (MH SCORE SENTIMENT)
potong = 20000
count = 0
with open(path2, 'r') as file_csv:
    csv_reader = csv.reader(file_csv)
    next(csv_reader)
    for baris in csv_reader:
        text = re.sub(r'[-.,:;!?\n@#]|//t', ' ', baris[6])
        text = ' '.join([word.strip(string.punctuation) for word in text.split() if word.lower() not in stop_words])

        data = list(ast.literal_eval(baris[21]).values())
        label = data[:-1]
        l = np.argmax(label)

        if label[l] > 0.95:
            if l == 0:
                score_neg.append([1, 0, 0])
                kalimat_neg.append(text)
                jumlah[0] += 1
            elif l == 1:
                score_neu.append([0, 1, 0])
                kalimat_neu.append(text)
                jumlah[1] += 1
            elif l == 2:
                score_pos.append([0, 0, 1])
                kalimat_pos.append(text)
                jumlah[2] += 1
            count += 1
        else:
            pass
        if count >= potong:
            break

print("Second dataset processed:", jumlah)

# Additional neutral data
with open(path3, "r") as file:
    reader = file.readlines()
    for text in reader:
        text = re.sub(r'[-.,:;!?\n@#]|//t', ' ', text)
        text = ' '.join([word.strip(string.punctuation) for word in text.split() if word.lower() not in stop_words])
        kalimat_neu.append(text)
        score_neu.append([0, 1, 0])
        jumlah[1] += 1

print("Additional neutral data processed:", jumlah)

# Additional negative data
with open(path4, "r") as file:
    reader = file.readlines()
    for text in reader:
        text = re.sub(r'[-.,:;!?\n@#]|//t', ' ', text)
        text = ' '.join([word.strip(string.punctuation) for word in text.split() if word.lower() not in stop_words])
        kalimat_neg.append(text)
        score_neg.append([1, 0, 0])
        jumlah[2] += 1

print("Additional negative data processed:", jumlah)

# Preprocess tweet sentiment extraction dataset
for j in [path5, path6]:
    with open(j, "r") as data:
        for h in data:
            try:
                fil = json.loads(h)

                lab = (fil["label"])
                if lab == 0:
                    score_neg.append([1, 0, 0])
                    jumlah[0] += 1
                    text = re.sub(r'[-.,:;!?\n@#]|//t', ' ', fil["text"])
                    text = ' '.join([word.strip(string.punctuation) for word in text.split() if word.lower() not in stop_words])
                    kalimat_neg.append(text)
                elif lab == 1:
                    score_neu.append([0, 1, 0])
                    jumlah[1] += 1
                    text = re.sub(r'[-.,:;!?\n@#]|//t', ' ', fil["text"])
                    text = ' '.join([word.strip(string.punctuation) for word in text.split() if word.lower() not in stop_words])
                    kalimat_neu.append(text)
                elif lab == 2:
                    score_pos.append([0, 0, 1])
                    jumlah[2] += 1
                    text = re.sub(r'[-.,:;!?\n@#]|//t', ' ', fil["text"])
                    text = ' '.join([word.strip(string.punctuation) for word in text.split() if word.lower() not in stop_words])
                    kalimat_pos.append(text)
            except:
                print("error : ", h)

print("Tweet sentiment extraction dataset processed:", jumlah)

# Augmentation
jumlah_kelas_terbesar = max(jumlah)

for _ in range(jumlah_kelas_terbesar - len(kalimat_neg)):
    random_index = random.randint(0, len(kalimat_neg) - 1)
    original_text = kalimat_neg[random_index]
    augmented_text = aug.RandomWordAug().augment(original_text)
    kalimat_neg.append(augmented_text)
    score_neg.append([1, 0, 0])
    jumlah[0] += 1

for _ in range(jumlah_kelas_terbesar - len(kalimat_neu)):
    random_index = random.randint(0, len(kalimat_neu) - 1)
    original_text = kalimat_neu[random_index]
    augmented_text = aug.RandomWordAug().augment(original_text)
    if isinstance(augmented_text, list):
        augmented_text = ' '.join(augmented_text)
    kalimat_neu.append(augmented_text)
    score_neu.append([0, 1, 0])
    jumlah[1] += 1

for _ in range(jumlah_kelas_terbesar - len(kalimat_pos)):
    random_index = random.randint(0, len(kalimat_pos) - 1)
    original_text = kalimat_pos[random_index]
    augmented_text = aug.RandomWordAug().augment(original_text)
    kalimat_pos.append(augmented_text)
    score_pos.append([0, 0, 1])
    jumlah[2] += 1

print("Augmentation results:", jumlah)

# Combine all data
full = []
for i in range(jumlah_kelas_terbesar):
    full.append([kalimat_neg[i], score_neg[i]])
    full.append([kalimat_neu[i], score_neu[i]])
    full.append([kalimat_pos[i], score_pos[i]])

# Write data to CSV
with open(tulis_path, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    for row in full:
        writer.writerow(row)

print("Data written to:", tulis_path)


First dataset processed: [799999, 0, 800000]
Second dataset processed: [799999, 20000, 800000]
Additional neutral data processed: [799999, 25458, 800000]
Additional negative data processed: [799999, 25458, 801058]
Tweet sentiment extraction dataset processed: [808781, 38006, 810743]
Augmentation results: [809685, 810743, 811801]
Data written to: /content/drive/MyDrive/DATASET_CAPSTONE/full_AUG.csv


In [ ]:
print(full[:10])
print(full[-10:])

[["upset that he can't update his Facebook by texting it might cry as result School today also Blah", [1, 0, 0]], ["Today last day OCD Awareness Week Read about what OCD what it isn't OCDAwarenessWeek https co/BL0zfAYMsN OCDweek OCD mentalhealth SocialWorkTwitter socialwork", [0, 1, 0]], ['I LOVE Health4UandPets u guys r best', [0, 0, 1]], ['Kenichan I dived many times ball Managed save 50 rest go out bounds', [1, 0, 0]], ['It was miraculous I immediately felt difference Hear how BrainsWay Deep TMS treatment helped Cathy reclaim her life from OCD after 30 years intensive treatments various medications https co/fx4YcUwuRx OCDWeek DeepTMS', [0, 1, 0]], ['im meeting up one my besties tonight Cant wait GIRL TALK', [0, 0, 1]], ['my whole body feels itchy like its fire', [1, 0, 0]], ['It IT DOES 😯😯😯📚📖 OCDAwarenessWeek 🙏🙏🙏🙏🙏🙏🙏🙏🙏 My gosh it means SO much It took FIVE years from first word publication day Thank you saying that OCD OCDweek https co/p6rA8Eyeep', [0, 1, 0]], ['DaRealSunisaKim Than

In [1]:
!pip install googletrans==4.0.0-rc1
!pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.7 MB/s eta 0:00:00


In [2]:
from googletrans import Translator

def terjemahkan_kalimat(kalimat, tujuan_bahasa='en'):
    translator = Translator()
    terjemahan = translator.translate(kalimat, dest=tujuan_bahasa)
    return terjemahan.text

if __name__ == "__main__":
    kalimat_input = input("Masukkan kalimat: ")
    bahasa_tujuan = "id"

    hasil_terjemahan = terjemahkan_kalimat(kalimat_input, bahasa_tujuan)
    print(f"Terjemahan: {hasil_terjemahan}")


Masukkan kalimat: helo
Terjemahan: Halo
